In [ ]:
from fastapi import FastAPI, HTTPException, Depends, Form
import time
from tortoise.contrib.fastapi import HTTPNotFoundError, register_tortoise
import logging
from botmiddlewares import ChannelCheckMiddleware
from aiogram import Bot, Dispatcher, types
from aiogram.types import InlineKeyboardButton, WebAppInfo, InlineKeyboardMarkup, BotCommand
from connectToDatabase import connectToDatabase
from models import User
from schemas import UpdateGamePts
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from pydantic import BaseModel
from typing import Optional
from jose import JWTError, jwt
from datetime import datetime, timedelta


PINNED_IMAGE_URL = 'https://github.com/Ambrose280/GameWare/blob/master/Screenshot%20(2).png'
TOKEN = '7368408525:AAHlC917Pv9mPzjIRtb_AieOBStZy4IofOo'
CHANNEL_USERNAME = 'ifioktestchannel'
WEBHOOK_PATH = f"/bot/{TOKEN}"
RENDER_WEB_SERVICE_NAME = "8230-105-113-40-56"
WEBHOOK_URL = "https://" + RENDER_WEB_SERVICE_NAME + ".ngrok-free.app" + WEBHOOK_PATH

# Logging
logging.basicConfig(filemode='a', level=logging.INFO)

# Bot setup
bot = Bot(token=TOKEN)
dp = Dispatcher(bot=bot)
dp.middleware.setup(ChannelCheckMiddleware(CHANNEL_USERNAME))

# FastAPI setup
app = FastAPI()

# Tortoise ORM setup
register_tortoise(
    app,
    db_url="sqlite://db_bot.sqlite3",
    modules={"models": ["models"]},
    generate_schemas=True,
    add_exception_handlers=True,
)

# Keyboard markup
start_keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
start_keyboard.add(types.KeyboardButton(text="Menu"))

# Web app button
web_app_info = WebAppInfo(
    url="https://gigbot-swart.vercel.app/",
    name="Launch Game",
    with_launcher=False
)

# Function definitions
async def create_user(telegram_id, gamepts):
    await User.create(telegram_id=telegram_id, gamepts=gamepts)

async def update_user(user_id, gamepts, referral_count):
    user = await User.get(id=user_id)
    user.gamepts = gamepts
    user.referral_count = referral_count
    await user.save()

async def get_users():
    return await User.all()

# Message handlers
@dp.message_handler(commands=['start'])
async def start_handler(message: types.Message):
    user_full_name = message.from_user.full_name
    logging.info(f'Start: {message.from_user.id} {user_full_name} {time.asctime()}. Message: {message}')
    await message.reply(f"Hello, {user_full_name}!", reply_markup=start_keyboard)

    existing_user = await User.filter(telegram_id=message.from_user.id).first()
    if existing_user:
        logging.info('User already registered with bot.')
        return

    referrer_id = message.get_args()
    if referrer_id:
        logging.info(str(referrer_id))
        referrer = await User.filter(telegram_id=str(referrer_id)).first()
        if referrer and str(referrer.telegram_id) != str(message.from_user.id):
            referrer.referral_count += 1
            await referrer.save()
            await User.create(telegram_id=message.from_user.id, gamepts=0, referral_count=0)
            await bot.send_message(referrer.telegram_id, "You have received One Dino Point!")
            logging.info('User has been created, with existing referral')
        else:
            logging.info('Referrer does not exist or users cannot refer themselves')
    else:
        await User.create(telegram_id=message.from_user.id, gamepts=0, referral_count=0)
        logging.info('User has been created')

@dp.message_handler(lambda message: message.text == "Menu")
async def menu_handler(message: types.Message):
    user_full_name = message.from_user.full_name
    logging.info(f'Menu: {message.from_user.id} {user_full_name} {time.asctime()}. Message: {message}')
    await message.reply("Here's the menu:\n"
                        "/play - Option 1\n"
                        "/referral - Option 2\n"
                        "/option3 - Option 3")

@dp.message_handler(commands=['play'])
async def option1_handler(message: types.Message):
    user_full_name = message.from_user.full_name
    logging.info(f'Option 1: {message.from_user.id} {user_full_name} {time.asctime()}. Message: {message}')
    
    button = InlineKeyboardButton(text="Open Web App", web_app=web_app_info)
    keyboard = InlineKeyboardMarkup().add(button)
    await message.reply("You selected Option 1! Click the button below to launch the game.", reply_markup=keyboard)

@dp.message_handler(commands=['referral'])
async def option2_handler(message: types.Message):
    user_id = message.from_user.id
    referral_link = f"https://t.me/ifioktestbot?start={user_id}"
    await message.reply(f"Your referral link:\n{referral_link}")

@dp.message_handler(commands=['option3'])
async def option3_handler(message: types.Message):
    user_full_name = message.from_user.full_name
    await message.reply("You selected Option 3!")

# FastAPI event handlers
@app.on_event("startup")
async def on_startup():
    await connectToDatabase()
    webhook_info = await bot.get_webhook_info()
    if webhook_info.url != WEBHOOK_URL:
        await bot.set_webhook(url=WEBHOOK_URL)
    
    commands = [
        BotCommand(command="/start", description="Start the bot and see menu options"),
        BotCommand(command="/play", description="Select option 1"),
        BotCommand(command="/referral", description="Select option 2"),
        BotCommand(command="/option3", description="Select option 3"),
    ]
    await bot.set_my_commands(commands)

@app.post(WEBHOOK_PATH)
async def bot_webhook(update: dict):
    telegram_update = types.Update(**update)
    Dispatcher.set_current(dp)
    Bot.set_current(bot)
    await dp.process_update(telegram_update)

@app.on_event("shutdown")
async def on_shutdown():
    await bot.get_session().close()

@app.get("/")
def main_web_handler():
    return {"message": "Everything Okay!"}

DUMMY_USERS = {
    "user1": "password1",
    "user2": "password2",
    "user3": "password3"
}

# Secret key to encode and decode the JWT tokens
SECRET_KEY = "YOUR_SECRET_KEY"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30

# OAuth2 scheme
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")


# JWT Token creation function
def create_access_token(data: dict, expires_delta: Optional[timedelta] = None):
    to_encode = data.copy()
    if expires_delta:
        expire = datetime.utcnow() + expires_delta
    else:
        expire = datetime.utcnow() + timedelta(minutes=15)
    to_encode.update({"exp": expire})
    encoded_jwt = jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)
    return encoded_jwt

# Token validation function
async def get_current_user(token: str = Depends(oauth2_scheme)):
    credentials_exception = HTTPException(
        status_code=401,
        detail="Could not validate credentials",
        headers={"WWW-Authenticate": "Bearer"},
    )
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        username: str = payload.get("sub")
        if username is None or username not in DUMMY_USERS:
            raise credentials_exception
    except JWTError:
        raise credentials_exception
    return username

# Token endpoint
@app.post("/token")
async def login_for_access_token(form_data: OAuth2PasswordRequestForm = Depends()):
    username = form_data.username
    password = form_data.password
    if username not in DUMMY_USERS or DUMMY_USERS[username] != password:
        raise HTTPException(
            status_code=401,
            detail="Incorrect username or password",
            headers={"WWW-Authenticate": "Bearer"},
        )
    access_token_expires = timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    access_token = create_access_token(
        data={"sub": username}, expires_delta=access_token_expires
    )
    refresh_token = create_access_token(data={"sub": username})
    return {"access_token": access_token, "refresh_token": refresh_token, "token_type": "bearer"}

# Refresh token endpoint
@app.post("/refresh")
async def refresh_access_token(refresh_token: str = Form(...)):
    credentials_exception = HTTPException(
        status_code=401,
        detail="Could not validate credentials",
        headers={"WWW-Authenticate": "Bearer"},
    )
    try:
        payload = jwt.decode(refresh_token, SECRET_KEY, algorithms=[ALGORITHM])
        username: str = payload.get("sub")
        if username is None or username not in DUMMY_USERS:
            raise credentials_exception
    except JWTError:
        raise credentials_exception

    access_token_expires = timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    access_token = create_access_token(
        data={"sub": username}, expires_delta=access_token_expires
    )
    new_refresh_token = create_access_token(data={"sub": username})
    return {"access_token": access_token, "refresh_token": new_refresh_token, "token_type": "bearer"}

# Protected endpoint to get users
@app.get("/users/")
async def get_users(current_user: str = Depends(get_current_user)):
    return await User.all()

# Protected endpoint to update game points
@app.put("/user/{user_id}/gamepts")
async def update_gamepts(user_id: int, gamepts: int = Form(...), current_user: str = Depends(get_current_user)):
    try:
        user = await User.get(telegram_id=user_id)
        user.gamepts = gamepts
        await user.save()
        return user
    except Exception as e:
        raise HTTPException(status_code=404, detail=f"User with ID {user_id} not found")

